In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:

# 삼성전자 코드
code = '005930'
headers = {'User-Agent': 'Mozilla/5.0'}

# 수집할 날짜 범위
start_date = pd.to_datetime('2023-03-01')
end_date = pd.to_datetime('2024-02-28')

#모든 DataFrame을 출력할 수 있도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 데이터를 저장할 리스트
data = []

# 여러 페이지 돌면서 데이터 수집
for page in range(1, 70):  # 페이지 수를 넉넉히 설정 (필요시 조정)
    url = f"https://finance.naver.com/item/sise_day.nhn?code={code}&page={page}"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    table = soup.find('table', class_='type2')
    rows = table.find_all('tr')
    
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 6:
            date_str = cols[0].get_text(strip=True)
            if not date_str:
                continue  # 빈 줄 건너뛰기
            
            date = pd.to_datetime(date_str, format='%Y.%m.%d')
            close_price = cols[1].get_text(strip=True).replace(',', '')
            volume = cols[6].get_text(strip=True).replace(',', '')
            
            # 날짜가 범위 안에 있는지 확인
            if start_date <= date <= end_date:
                data.append({
                    '날짜': date.strftime('%Y.%m.%d'),
                    '종가': int(close_price),
                    '거래량': int(volume)
                })
            elif date < start_date:
                break  # 시작 날짜보다 과거 데이터면 중단

In [ ]:
# 데이터프레임으로 변환
df = pd.DataFrame(data).drop_duplicates(subset='날짜')
df = df.sort_values('날짜').reset_index(drop=True)

# 전일 종가 추가 (shift 사용)
df['전일종가'] = df['종가'].shift(1)

# 변동률 계산
df['변동률(%)'] = ((df['종가'] - df['전일종가']) / df['전일종가']) * 100
df['변동률(%)'] = df['변동률(%)'].fillna(0)  # 첫 행은 0으로

# 전일종가 컬럼 삭제
df = df.drop(columns=['전일종가'])

# 소수점 둘째자리까지만 반올림
df['변동률(%)'] = df['변동률(%)'].round(2)


# 결과 출력 
print(df)

# CSV 저장
df.to_csv('samsung_stock_20230301_20240228.csv', index=False, encoding='utf-8-sig')

             날짜          종가              거래량   변동률(%)
0    2023.03.02      60,800       13,095,682     0.0%
1    2023.03.03      60,500       10,711,405   -0.49%
2    2023.03.06      61,500       13,630,602    1.65%
3    2023.03.07      60,700       11,473,280    -1.3%
4    2023.03.08      60,300       14,161,857   -0.66%
5    2023.03.09      60,100       14,334,499   -0.33%
6    2023.03.10      59,500       11,902,471    -1.0%
7    2023.03.13      60,000       12,779,724    0.84%
8    2023.03.14      59,000       12,147,346   -1.67%
9    2023.03.15      59,800       10,482,149    1.36%
10   2023.03.16      59,900       10,611,939    0.17%
11   2023.03.17      61,300       14,090,110    2.34%
12   2023.03.20      60,200        9,618,009   -1.79%
13   2023.03.21      60,300        8,318,514    0.17%
14   2023.03.22      61,100        8,978,591    1.33%
15   2023.03.23      62,300       15,381,057    1.96%
16   2023.03.24      63,000       18,278,602    1.12%
17   2023.03.27      62,100 